In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Concatenate, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import files

# Загрузка файла CSV
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
# Чтение данных из CSV-файла с различными кодировками
try:
    df = pd.read_csv(file_name, header=None, names=['id', 'initial_situation', 'first_transition_situation', 'first_transition_emotion', 'second_transition_situation', 'second_transition_emotion'], encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_name, header=None, names=['id', 'initial_situation', 'first_transition_situation', 'first_transition_emotion', 'second_transition_situation', 'second_transition_emotion'], encoding='cp1252')
    except UnicodeDecodeError:
        df = pd.read_csv(file_name, header=None, names=['id', 'initial_situation', 'first_transition_situation', 'first_transition_emotion', 'second_transition_situation', 'second_transition_emotion'], encoding='utf-16')

# Просмотр первых строк данных
df.head()

# Чтение данных из CSV-файла
df = pd.read_csv('situatsiya1.csv')

# Подготовка данных
X = df[['initial_situation', 'first_transition_situation', 'second_transition_situation']]
y = df[['first_transition_emotion', 'second_transition_emotion']]

# Объединение меток эмоций для мульти-меточной классификации
y_combined = y.apply(lambda x: [x['first_transition_emotion'], x['second_transition_emotion']], axis=1)

# Кодирование эмоций с помощью MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(y_combined)

# Токенизация текста
max_words = 1000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(pd.concat([X['initial_situation'], X['first_transition_situation'], X['second_transition_situation']]))

# Преобразование текстов в последовательности чисел и их дополнение до одинаковой длины
X_initial = pad_sequences(tokenizer.texts_to_sequences(X['initial_situation']), maxlen=max_len)
X_first = pad_sequences(tokenizer.texts_to_sequences(X['first_transition_situation']), maxlen=max_len)
X_second = pad_sequences(tokenizer.texts_to_sequences(X['second_transition_situation']), maxlen=max_len)

# Разделение данных на обучающую и тестовую выборки
X_train_initial, X_test_initial, X_train_first, X_test_first, X_train_second, X_test_second, y_train, y_test = train_test_split(
    X_initial, X_first, X_second, y_encoded, test_size=0.2, random_state=42
)

# Определение входов для модели
input_initial = Input(shape=(max_len,))
input_first = Input(shape=(max_len,))
input_second = Input(shape=(max_len,))

# Слои Embedding для каждого входа с L2-регуляризацией
embed_initial = Embedding(max_words, 32, input_length=max_len, embeddings_regularizer=l2(0.01))(input_initial)
embed_first = Embedding(max_words, 32, input_length=max_len, embeddings_regularizer=l2(0.01))(input_first)
embed_second = Embedding(max_words, 32, input_length=max_len, embeddings_regularizer=l2(0.01))(input_second)

# Объединение входов
merged = Concatenate()([embed_initial, embed_first, embed_second])

# LSTM слой с L2-регуляризацией и Dropout
lstm = LSTM(64, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))(merged)
dropout1 = Dropout(0.5)(lstm)

# Dense слой с ReLU активацией и L2-регуляризацией
dense = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(dropout1)
dropout2 = Dropout(0.5)(dense)

# Выходной слой с Sigmoid активацией для многоклассовой классификации
output = Dense(len(mlb.classes_), activation='sigmoid')(dropout2)

# Создание и компиляция модели
model = Model(inputs=[input_initial, input_first, input_second], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Преобразование входных данных в тензоры
X_train_initial = tf.convert_to_tensor(X_train_initial, dtype=tf.int32)
X_train_first = tf.convert_to_tensor(X_train_first, dtype=tf.int32)
X_train_second = tf.convert_to_tensor(X_train_second, dtype=tf.int32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

# Определение callback для ранней остановки обучения
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Обучение модели
history = model.fit(
    [X_train_initial, X_train_first, X_train_second],
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Функция предсказания эмоций с вероятностями
def predict_emotions(text):
    text_seq = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=max_len)
    prediction = model.predict([text_seq, text_seq, text_seq])
    emotion_probabilities = {mlb.classes_[i]: prob for i, prob in enumerate(prediction[0])}
    return emotion_probabilities

# Пример анализа ситуации
text = "Я так рад, что наконец-то закончил этот сложный проект, но теперь чувствую себя совершенно истощенным."
predicted_emotions = predict_emotions(text)
print(f"Результат анализа: {predicted_emotions}")

Saving situatsiya1.csv to situatsiya1 (2).csv
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 171ms/step - accuracy: 0.1002 - loss: 3.8098 - val_accuracy: 0.0417 - val_loss: 3.3850
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.1943 - loss: 3.2646 - val_accuracy: 0.0417 - val_loss: 2.9113
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.0754 - loss: 2.8130 - val_accuracy: 0.0417 - val_loss: 2.5185
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.1347 - loss: 2.4340 - val_accuracy: 0.0417 - val_loss: 2.1817
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.1639 - loss: 2.1050 - val_accuracy: 0.0417 - val_loss: 1.8584
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.1957 - loss: 1.8111 - val_accuracy: 0.0417 - val_loss: 1.5971
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.1876 - loss: 1.5668 - val_accuracy: 0.0417 - val_loss: 1.4039
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.1717 - loss: 1.4156 - val_accuracy: 0.0417 - val_loss: 1.2499
Epoch 9/5

In [1]:
from graphviz import Digraph

# Создание объекта графа
dot = Digraph()

# Импорт библиотек
dot.node('A', 'Импорт библиотек')
dot.node('B', 'Загрузка файла CSV')
dot.node('C', 'Чтение данных из CSV-файла с различными кодировками')
dot.node('D', 'Просмотр первых строк данных')
dot.node('E', 'Чтение данных из другого CSV-файла')
dot.node('F', 'Подготовка данных')
dot.node('G', 'Кодирование эмоций')
dot.node('H', 'Токенизация текста')
dot.node('I', 'Преобразование текстов в последовательности чисел')
dot.node('J', 'Разделение данных на обучающую и тестовую выборки')
dot.node('K', 'Определение входов для модели')
dot.node('L', 'Создание слоев Embedding')
dot.node('M', 'Объединение входов')
dot.node('N', 'LSTM слой с L2-регуляризацией и Dropout')
dot.node('O', 'Dense слой с ReLU активацией и L2-регуляризацией')
dot.node('P', 'Выходной слой с Sigmoid активацией')
dot.node('Q', 'Создание и компиляция модели')
dot.node('R', 'Преобразование входных данных в тензоры')
dot.node('S', 'Определение callback для ранней остановки обучения')
dot.node('T', 'Обучение модели')
dot.node('U', 'Функция предсказания эмоций с вероятностями')
dot.node('V', 'Пример анализа ситуации')

# Добавление ребер
dot.edges(['AB', 'BC', 'CD', 'DE', 'EF', 'FG', 'GH', 'HI', 'IJ', 'JK', 'KL', 'LM', 'MN', 'NO', 'OP', 'PQ', 'QR', 'RS', 'ST', 'TU', 'UV'])

# Сохранение и отображение графа
dot.render('code_flowchart', format='png', view=True)

'code_flowchart.png'